<a href="https://colab.research.google.com/github/rajeevrpandey/California-Housing-Prices-Prediction/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# artificially growing the training set, called data augmentation or training set expansion.

In [2]:
# initial setup

import sys

assert sys.version_info >= (3, 7)


from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")


import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)


from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

X, y = mnist.data, mnist.target

# 10000 for testing
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
# Write a function that can shift an MNIST image in any direction (left, right, up, or down) by one pixel. You can use the shift() function from the scipy.ndimage module.
# Then, for each image in the training set, create four shifted copies (one per direction) and add them to the training set.
# Finally, train your best model on this expanded training set and measure its accuracy on the test set.

In [5]:
from scipy.ndimage import shift

# the following function shifts the image dy pixels down and dx pixel to the right, for shifting left or up use negative values
def shift_image(image, dx, dy): #  image (the input image), dx (the shift along the x-axis), and dy (the shift along the y-axis)
    image = image.reshape((28, 28)) #  input image is reshaped into a 28x28 two-dimensional array
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant") # The cval=0 parameter specifies the constant value to use for out-of-bounds points,
    # and the mode="constant" parameter indicates that values outside the boundaries of the input are filled with the constant value.
    return shifted_image.reshape([-1]) #  resulting shifted image is reshaped back into a 1D array using the reshape method

In [6]:
# first add the original X_train and y_train
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

In [7]:
# now add the shifted X_train but y_train will be the original ones
for dx, dy in ((-1, 0), (1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):  # zip function pairs corresponding elements from X_train and y_train
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

In [8]:
# The resulting X_train_augmented and y_train_augmented NumPy arrays represent
# the augmented training sets, which contain the original training data as well
# as the additional images created by shifting, along with their corresponding
# labels.
import numpy as np

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [9]:
# Let's shuffle the augmented training set, or else all shifted images will be grouped together:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [10]:
# Now let's train the model using the best hyperparameters found by training on 10K entries:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5, 6]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5)
grid_search.fit(X_train[:10_000], y_train[:10_000])



knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [11]:
augmented_accuracy = knn_clf.score(X_test, y_test)
augmented_accuracy

0.9763

In [12]:
# accuracy boost